## Setup
### Project setup

In [ ]:
if run_init:
    %pip install -U pip
    !if  [ ! -d "deep-learning-project" ] ; then git clone https://github.com/albertsgarde/deep-learning-project.git; fi
    !cd deep-learning-project && git reset --hard && git pull
    !source deep-learning-project/setup.sh deep-learning-project
run_init = False

In [ ]:
run_init = True

In [ ]:
run_init = False

### Imports

In [ ]:
import numpy as np
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as nn_func
import torch.optim as optim
from torch.autograd import Variable

import audio_samples_py as aus

### Device setup

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Running GPU.") if use_cuda else print("No GPU available.")

## Data
### Parameters

In [ ]:
SAMPLE_LENGTH = 256

### Generation

In [ ]:
import warnings

class AudioDataSet(torch.utils.data.Dataset):
    def __init__(self, parameters: aus.DataParameters):
         self.parameters = parameters

    def __len__(self):
        return np.iinfo(np.int64).max
    
    def __getitem__(self, index):
        data_point = self.parameters.generate_at_index(index)
        return data_point.get_samples(), torch.tensor([data_point.get_frequency_map()]).unsqueeze(0)



In [ ]:

data_loader_params = {"batch_size": 64}

parameters = aus.DataParameters(num_samples=SAMPLE_LENGTH).add_sine((0.5,0.75))
training_parameters = parameters.with_seed_offset(0)
training_generator = aus.DataGenerator(training_parameters)
training_loader = torch.utils.data.DataLoader(AudioDataSet(training_parameters), **data_loader_params)
validation_parameters = parameters.with_seed_offset(1)
validation_generator = aus.DataGenerator(validation_parameters)
validation_loader = torch.utils.data.DataLoader(AudioDataSet(validation_parameters), **data_loader_params)




## Neural Network

### Network Parameters

In [ ]:
conv_depth = 2
channels = [8, 8, 8, 8]
kernel_sizes = [5, 5, 5, 5]
strides = [1 for _ in range(conv_depth)]
paddings = [int((kernel_size - 1)/2) for kernel_size in kernel_sizes]
poolings = [2,2,2,2]
conv_batch_norms = [False for _ in range(conv_depth)]
conv_dropouts = [0.0 for _ in range(conv_depth)]


lin_depth = 2
features = [256, 128]
lin_batch_norms = [False for _ in range(lin_depth)]
lin_dropouts = [0.0 for _ in range(lin_depth)]

### Parameter validation and processing

In [ ]:
assert len(channels) >= conv_depth
assert len(kernel_sizes) >= conv_depth
assert len(strides) >= conv_depth
assert len(paddings) >= conv_depth
assert len(poolings) >= conv_depth
assert len(conv_batch_norms) >= conv_depth
assert len(conv_dropouts) >= conv_depth
for kernel_size in kernel_sizes:
    assert kernel_size % 2 == 1, "Only odd kernel sizes are supported."
for dropout in conv_dropouts:
    assert 0 <= dropout and dropout <= 1, "Dropout must be between 0 and 1."

conv_size = []
input_size = SAMPLE_LENGTH
for i in range(conv_depth):
    conv_dim_reduction = kernel_sizes[i]-1-paddings[i]*2
    assert (input_size - conv_dim_reduction) % (strides[i]*poolings[i]) == 0
    conv_size.append(int((input_size - conv_dim_reduction)/strides[i]/poolings[i]))
    input_size = conv_size[i]
    print("Conv layer {} has output size {} and {} channels.".format(i, conv_size[i], channels[i]))

conv_output_size = conv_size[-1]*channels[-1]


assert len(features) >= lin_depth
assert len(lin_batch_norms) >= lin_depth
assert len(lin_dropouts) >= lin_depth
for dropout in lin_dropouts:
    assert 0 <= dropout and dropout <= 1, "Dropout must be between 0 and 1."
for i in range(lin_depth):
    print("Lin layer {} has output size {}.".format(i, features[i]))

### Network

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.convs = nn.ModuleList()
        in_channels = 1
        for i in range(conv_depth):
            conv = nn.Conv1d(in_channels=in_channels, out_channels=channels[i], kernel_size=kernel_sizes[i], stride=strides[i], padding=paddings[i])
            in_channels = channels[i]
            pool = nn.MaxPool1d(poolings[i])
            batchnorm = nn.BatchNorm1d(channels[i]) if (conv_batch_norms[i]) else nn.Identity()
            dropout = nn.Dropout(p=conv_dropouts[i])

            self.convs.append(nn.ModuleList([conv, pool, batchnorm, dropout]))
        
        self.lins = nn.ModuleList()
        in_features = conv_output_size
        for i in range(lin_depth):
            lin = nn.Linear(in_features=in_features, out_features=features[i])
            in_features = features[i]
            batchnorm = nn.BatchNorm1d(features[i]) if (lin_batch_norms[i]) else nn.Identity()
            dropout = nn.Dropout(p=lin_dropouts[i])

            self.lins.append(nn.ModuleList([lin, batchnorm, dropout]))

        self.lin_out = nn.Linear(in_features=in_features, out_features=1)
        
        
    def forward(self, x):
        x = x.unsqueeze(1)

        for conv, pool, batchnorm, dropout in self.convs:
            x = conv(x)
            x = pool(x)
            x = nn_func.relu(x)
            x = batchnorm(x)
            x = dropout(x)

        x = x.flatten(1)

        for lin, batchnorm, dropout in self.lins:
            x = lin(x)
            x = nn_func.relu(x)
            x = batchnorm(x)
            x = dropout(x)
            
        return self.lin_out(x).unsqueeze(1)

net = Net()
if use_cuda:
    net.cuda()
print(net)

## Training

In [ ]:
LEARNING_RATE = 1e-2
WEIGHT_DECAY = 1e-5

In [ ]:
criterion = nn.MSELoss()  

# weight_decay is equal to L2 regularization
optimizer = optim.AdamW(net.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY) 


In [ ]:
def to_torch(x):
    variable = Variable(torch.from_numpy(x))
    if use_cuda:
        variable = variable.cuda()
    return variable

def test_net(net, validation_loader, criterion):
    was_training = net.training
    net.eval()
    num_iterations = 100
    total_loss = 0
    for data_point, frequency in itertools.islice(validation_loader, num_iterations):
        data_point = data_point.to(device)
        frequency = frequency.to(device)
        output = net(data_point)
        total_loss += criterion(output, frequency)
    net.train(mode=was_training)
    return total_loss/num_iterations

def manual_test(net, validation_generator):
    was_training = net.training
    net.eval()
    num_iterations = 5
    for data_point in validation_generator.next_n(num_iterations):
        samples = to_torch(data_point.get_samples()).unsqueeze(0)
        freq_map = to_torch(np.array([data_point.get_frequency_map()])).unsqueeze(0)
        output = net(samples)
        target_frequency = parameters.map_to_frequency(freq_map.item())
        output_frequency = parameters.map_to_frequency(output.item())
        print("Frequency: {:.2f} Output: {:.2f} Cent diff: {:.2f}".format(target_frequency, output_frequency, aus.cent_diff(target_frequency, output_frequency)))
    net.train(mode=was_training)



In [ ]:
NUM_BATCHES = 2000
EVAL_EVERY = 200


manual_test(net, validation_generator)

net.train()

for i, (data_point, frequency) in enumerate(itertools.islice(training_loader, NUM_BATCHES)):
    if i%EVAL_EVERY == 0:
        print(f"Loss at iteration {i}: {test_net(net, validation_loader, criterion)}")
    data_point = data_point.to(device)
    frequency = frequency.to(device)
    output = net(data_point)
    loss = criterion(output, frequency)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f"Loss at iteration {NUM_BATCHES}: {test_net(net, validation_loader, criterion)}")
manual_test(net, validation_generator)
    